In [18]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [21]:
item = 66841
url = 'http://www9.health.gov.au/mbs/search.cfm?'
payload = {'q': '{}'.format(str(item)),
          'sopt': 'I'}
r = requests.get(url, params=payload)
# r = requests.get('http://www9.health.gov.au/mbs/search.cfm?q={}&sopt=S'.format(str(item)))
soup = BeautifulSoup(r.text, 'html.parser')

In [24]:
type(soup)

bs4.BeautifulSoup

In [22]:
print(soup.prettify())

<!--[if HTML5]><![endif]-->
<!DOCTYPE html>
<!--[if lt IE 7 ]><html lang="en-au" class="ie-6"><![endif]-->
<!--[if IE 7 ]><html lang="en-au" class="ie-7"><![endif]-->
<!--[if IE 8 ]><html lang="en-au" class="ie-8"><![endif]-->
<!--[if (gte IE 9)|!(IE)]>
<!-->
<html lang="en-au">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1.0" name="viewport"/>
  <!-- Core CSS -->
  <link href="http://www.health.gov.au/internet/r/publishing.nsf/AttachmentsByTitle/bootstrap.css/$FILE/bootstrap.no-icons.css?OpenElement&amp;v=1.0" rel="stylesheet"/>
  <link href="http://fonts.googleapis.com/css?family=Open+Sans" rel="stylesheet"/>
  <link href="http://www.health.gov.au/internet/r/publishing.nsf/AttachmentsByTitle/font-awesome.css/$FILE/font-awesome.css" rel="stylesheet"/>
  <link href="http://www.health.gov.au/internet/r/publishing.nsf/AttachmentsByTitle/dha.css/$FILE/dha.css?OpenElement&amp;v=1.0" rel="stylesheet"/>
  <link href="http://www.health

In [23]:
def get_categories(soup):
    """Extract the categories from the soup."""
    categories = []
    for elem in soup.find_all('h3'):
        if 'category' in elem.text.lower():
            categories.append(elem.text.split('<')[0])
    return categories

categories = get_categories(soup)
print('Categories: {}'.format(categories))

Categories: [u'Category 6 - PATHOLOGY SERVICES']


In [8]:
def get_info(soup):
    """Extract group, subgroup and subheading from the soup."""
    group = None
    subgroup = None
    subheading = None
    out = []
    for elem in soup.find_all('div'):
        elem_class = elem.get('class')
        if (elem_class is not None) and (u'span9' in elem_class):
            out.append(elem.text)
    group = out[0]
    if len(out) > 1: subgroup = out[1]
    if len(out) > 2: subheading = out[2]
    return group, subgroup, subheading

group, subgroup, subheading = get_info(soup)
print('Group: {}'.format(group))
print('Subgroup: {}'.format(subgroup))
print('Subheading: {}'.format(subheading))

Group: P2 - Chemical
Subgroup: None
Subheading: None


In [13]:
def get_description(soup):
    """Extract the item description from the soup."""
    description = None
    for elem in soup.find_all('p'):
        if elem.get('align') =='justify':
            description = elem.text.encode('utf-8').strip()
    return str(description)

description = get_description(soup)
print('Description: {}'.format(description))

Description: Quantitation of HbA1c (glycated haemoglobin) performed for the diagnosis of diabetes in asymptomatic patients at high risk.  (Item is subject to rule 25)


In [14]:
def get_dates(soup):
    """Extract the relevant dates from the soup."""
    dates_keys = []
    dates_values = []
    for elem in soup.find_all('p'):
        if 'date' in elem.text.lower():
            for i, div_elem in enumerate(elem.find_all('div')):
                if u'span8' in div_elem.get('class'):
                    dates_keys.append(div_elem.text)
                elif u'span4' in div_elem.get('class'):
                    dates_values.append(div_elem.text)
    dates = {k: v for k, v in zip(dates_keys, dates_values)}
    return dates

dates = get_dates(soup)
print('Relevant dates: {}'.format(dates))

Relevant dates: {u'Schedule Fee Start Date:': u'01-Nov-2014', u'Description Start Date:': u'01-Nov-2014', u'Item Start Date:': u'01-Nov-2014'}


In [15]:
def get_fees(soup):
    """Extract the fees from the soup."""
    fee = None
    benefit75 = None
    benefit85 = None
    safety_net = None
    for elem in soup.find_all('p'):
        for elem_p in elem.find_all('p'):
            if '$' in elem_p.text:
                splitted = np.array(re.sub(' +', ' ', elem_p.text).split(' '))
                if u'Fee:' in splitted:
                    fees = filter(lambda x: '$' in x, splitted) 
                    fee = fees[0]
                    benefit75 = fees[1]
                    if len(fees) > 2: benefit85 = fees[2] # this is not always there
                elif u'Safety' in splitted:
                    safety_net = splitted[-1]
    return fee, benefit75, benefit85, safety_net

fee, benefit75, benefit85, safety_net = get_fees(soup)

print('Fee: {}'.format(fee))
print('Benefit 75%: {}'.format(benefit75))
print('Benefit 85%: {}'.format(benefit85))
print('Safety net: {}'.format(safety_net))

Fee: $16.80
Benefit 75%: $12.60
Benefit 85%: $14.30
Safety net: None
